# Credential Success Report (BMC Discovery)

This notebook reproduces the DisMAL `credential_success` report using the REST API.
It reads connection details from `config.yaml`, executes Discovery queries,
and assembles a CSV matching the CLI’s headers and formatting.

> **NOTE:** This can take a little while to run if you have lots of DiscoveryAccesses

## Requirements

We use `requests` for HTTP, `pandas` for tabular data, and `PyYAML` to read configuration.
Uncomment the following to install them in your environment.

In [ ]:
# %pip install -q requests pandas pyyaml

import pandas as pd
import requests
import yaml
from pathlib import Path
from urllib.parse import urljoin
import ipaddress
import json, os
import math
import tideway

## Configuration (from config.yaml)

Reads settings from `../config.yaml` including target, token/token_file,
API version, and SSL verification preference.
Saves the CSV to `../output_<target>/credential_success.csv`.

In [ ]:
from pathlib import Path
import yaml

def load_config_params(
    start: Path,
    appliance_name: str = None,
    appliance_index: int = 0,
) -> dict:
    def _find_repo_root(start: Path) -> Path:
        for p in [start] + list(start.parents):
            if (p / 'config.yaml').exists():
                return p
        return start.parent

    repo_root = _find_repo_root(start)
    config_path = repo_root / 'config.yaml'

    with open(config_path, 'r') as fh:
        cfg = yaml.safe_load(fh) or {}

    apps = cfg.get('appliances') or []
    selected = None
    if isinstance(apps, list) and apps:
        if appliance_name:
            selected = next((a for a in apps if a.get('name') == appliance_name), None)
            if selected is None:
                raise ValueError(f"No appliance named '{appliance_name}' in config.yaml")
        else:
            try:
                selected = apps[int(appliance_index)]
            except Exception:
                selected = apps[0]

    target = ((selected or {}).get('target') or cfg.get('target') or '').strip()
    if not target:
        raise ValueError('config.yaml missing "target"')

    token = (((selected or {}).get('token') or cfg.get('token') or '').strip())
    token_file = (selected or {}).get('token_file') or cfg.get('token_file') or cfg.get('f_token')
    if not token and token_file:
        tf_path = Path(token_file)
        if not tf_path.is_absolute():
            tf_path = repo_root / tf_path
        with open(tf_path, 'r') as tf:
            token = tf.read().strip()
    if not token:
        raise ValueError('API token not found in config.yaml (token or token_file)')

    api_version = str((selected or {}).get('api_version') or cfg.get('api_version') or 'v1.14')
    verify_ssl = bool((selected or {}).get('verify_ssl', cfg.get('verify_ssl', True)))

    sanitized = target.replace('.', '_').replace(':', '_').replace('/', '_')
    output_dir = repo_root / f'output_{sanitized}'
    output_dir.mkdir(parents=True, exist_ok=True)

    return {
        "repo_root": repo_root,
        "config_path": config_path,
        "cfg": cfg,
        "selected": selected,
        "target": target,
        "token": token,
        "api_version": api_version,
        "verify_ssl": verify_ssl,
        "output_dir": output_dir,
    }

In [ ]:
def init_appliance(appliance_name: str = "prod"):
    params = load_config_params(Path.cwd(), appliance_name=appliance_name)

    target = params["target"]
    api_version = params["api_version"]
    verify_ssl = params["verify_ssl"]
    output_dir = params["output_dir"]

    print('Base Host     :', target)
    print('API Version   :', api_version)
    print('Verify SSL    :', verify_ssl)
    print('Output folder :', output_dir)

    api_number = api_version.lstrip('v')
    app = tideway.appliance(target, params["token"], api_version=api_number, ssl_verify=verify_ssl)

    try:
        about = app.api_about
        print('Appliance reachable:', about.status_code)
    except Exception as e:
        print('Warning: failed to contact appliance /api/about:', e)

    return {
        "params": params,
        "target": target,
        "app": app,
        "api_version":api_number,
        "output_dir":output_dir,
    }

# Initialise Instances

In [ ]:
print("Initialise Prod:")
twprod = init_appliance("prod")

print("Initialise Dev:")
twdev = init_appliance("dev")

In [ ]:
# bootstrapping variables

token = twprod["params"]["token"]
tokendev = twdev["params"]["token"]
VERIFY_SSL = twprod["params"]["verify_ssl"]
verify_ssl_dev = twdev["params"]["verify_ssl"]
target = twprod["target"]
target_dev = twdev["target"]
api_version = twprod["api_version"]
api_version_dev = twdev["api_version"]
output_dir = twprod["output_dir"]
output_dir_dev = twdev["output_dir"]

BASE_URL = target if ('://' in target) else f'https://{target}'
API_VERSION = api_version

## Fetch reference data

- Vault credentials (labels, usernames, ranges, enabled/usage)
- Scan ranges and Exclude ranges
- Outpost list and (credential -> outpost) mappings

In [ ]:
# Vault credentials
prodvault = twprod["app"].credentials()
devvault = twdev["app"].credentials()

print(twprod['target'])
vault_prod = prodvault.get_vault_credentials.json()
df = pd.DataFrame(vault_prod)
display(df.head(10))

print(twdev['target'])
vault_dev = devvault.get_vault_credentials.json()
df = pd.DataFrame(vault_dev)
display(df.head(10))

In [ ]:
def get_results(instance, qry, columns=['No Results']):
    results = instance['app'].data().search({'query': qry}, format='object', limit=500)
    df = pd.DataFrame(results) if results else pd.DataFrame()
    if df.empty:
        # Provide headers if no rows returned
        df = pd.DataFrame(columns=columns)
    if 'Discovery Instance' not in df.columns:
        df.insert(0, 'Discovery Instance', instance['target'])
    else:
        df['Discovery Instance'] = instance['target']
    return df

# Scan ranges and excludes
qry_scanrange = '''
            search ScanRange where scan_type = 'Scheduled' show range_id as 'ID', label as 'Label',
            (range_strings or provider) as 'Scan_Range', scan_level as 'Level',
            recurrenceDescription(schedule) as 'Date_Rules'
            '''

scan_prod = get_results(twprod,qry_scanrange,['ID', 'Label', 'Scan_Range', 'Level', 'Date_Rules'])
print(twprod['target'])
display(scan_prod.head(5))

scan_dev = get_results(twdev,qry_scanrange,['ID', 'Label', 'Scan_Range', 'Level', 'Date_Rules'])
print(twdev['target'])
display(scan_dev.head(5))

qry_excludes = '''
        search in '_System' ExcludeRange show exrange_id as 'ID', name as 'Label',
        range_strings as 'Scan_Range',recurrenceDescription(schedule) as 'Date_Rules'
        '''

ex_prod = get_results(twprod,qry_excludes,['ID', 'Label', 'Scan_Range', 'Level', 'Date_Rules'])
print(twprod['target'])
display(scan_prod.head(5))

ex_dev = get_results(twdev,qry_excludes,['ID', 'Label', 'Scan_Range', 'Level', 'Date_Rules'])
print(twdev['target'])
display(scan_dev.head(5))

In [ ]:
# Outposts list and mapping
def get_endpoint(instance, ep):
    results = instance['app'].get(ep)
    df = pd.DataFrame(results.json()) if results else pd.DataFrame()
    if df.empty:
        # Provide headers if no rows returned
        df = pd.DataFrame(columns=['name','id','url','scope','enabled','exclude_ranges','scan_ranges','version','last_contact'])
    df.insert(0, 'Discovery Instance', instance['target'])
    return df

outposts_prod = get_endpoint(twprod, '/discovery/outposts')
display(outposts_prod.head(5))

outposts_dev = get_endpoint(twdev, '/discovery/outposts')
display(outposts_dev.head(5))

In [ ]:
# We're going to run this again without countUnique as this can be intensive and cause the API to timeout
# Testing: search SessionResult where #id = 'node id'
qry_outpost_credentials = '''
search SessionResult
show credential, credential as 'uuid', outpost
'''
outpost_creds_prod = get_results(twprod,qry_outpost_credentials,["Discovery Instance","credential","uuid","outpost"]).drop_duplicates()
print(twprod['target'])
display(outpost_creds_prod.head(5))

In [ ]:
outpost_creds_dev = get_results(twdev,qry_outpost_credentials,["Discovery Instance","credential","uuid","outpost"]).drop_duplicates()
print(twprod['target'])
display(outpost_creds_dev.head(5))

In [ ]:
def build_credential_outpost_map(outposts_df, creds_df):
    # Map outpost IDs to URLs
    id_to_url = {}
    for _, row in outposts_df.iterrows():
        op_id = row.get("id") or row.get("outpost") or row.get("outpost_id") or row.get("uuid")
        if op_id:
            id_to_url[op_id] = row.get("url")

    # Map credentials to outpost info
    cred_outpost_map = {}
    for _, row in creds_df.iterrows():
        uuid = row.get("credential") or row.get("uuid")
        opid = row.get("outpost")
        if uuid and opid:
            info = {"id": opid, "url": id_to_url.get(opid)}
            cred_outpost_map[uuid.lower()] = info

    # Return DataFrame
    return pd.DataFrame.from_dict(cred_outpost_map, orient="index")
    
prod_map = build_credential_outpost_map(outposts_prod, outpost_creds_prod)
display(prod_map.head(10))
dev_map = build_credential_outpost_map(outposts_dev, outpost_creds_dev)
display(dev_map.head(10))

## Execute success/failure queries

We gather counts for all time and for the last 7 days, from SessionResult and DeviceInfo.

In [ ]:
# Success (all time)
qry_credential_success = '''
search SessionResult where success
show (credential or slave) as 'SessionResult.credential_or_slave',
     (credential or slave) as 'uuid',
     session_type as 'SessionResult.session_type',
     outpost as 'SessionResult.outpost'
processwith countUnique(1,0)
'''
cols= ["Discovery Instance","SessionResult.credential_or_slave","uuid","SessionResult.session_type","SessionResult.outpost","Count"]
suxCreds_prod = get_results(twprod, qry_credential_success,cols)
print(twprod['target'])
display(suxCreds_prod.head(5))
suxCreds_dev = get_results(twdev, qry_credential_success,cols)
print(twprod['target'])
display(suxCreds_dev.head(5))

In [ ]:
# DeviceInfo success (all time)
qry_deviceinfo_success = '''
search DeviceInfo where method_success
  and nodecount(traverse DiscoveryResult:DiscoveryAccessResult:DiscoveryAccess:DiscoveryAccess
                traverse DiscoveryAccess:Metadata:Detail:SessionResult) = 0
show (last_credential or last_slave) as 'DeviceInfo.last_credential',
     (last_credential or last_slave) as 'uuid',
     access_method as 'DeviceInfo.access_method'
process with countUnique(1,0)
'''
cols = ["DeviceInfo.last_credential","uuid","DeviceInfo.access_method","Count"]
suxDev_prod = get_results(twprod, qry_deviceinfo_success,cols)
print(twprod['target'])
display(suxDev_prod.head(5))
suxDev_dev = get_results(twdev, qry_deviceinfo_success,cols)
print(twdev['target'])
display(suxDev_dev.head(5))

In [ ]:
qry_credential_failure = '''
search SessionResult where not success
show (credential or slave) as 'SessionResult.credential_or_slave',
     (credential or slave) as 'uuid',
     session_type as 'SessionResult.session_type',
     outpost as 'SessionResult.outpost'
processwith countUnique(1,0)
'''
# Credential failure (all time)
cols = ["SessionResult.credential_or_slave","uuid","SessionResult.session_type","SessionResult.outpost","Count"]
failCreds_prod = get_results(twprod, qry_credential_failure,cols)
print(twprod['target'])
display(failCreds_prod.head(5))
failCreds_dev = get_results(twdev, qry_credential_failure,cols)
print(twdev['target'])
display(failCreds_dev.head(5))

In [ ]:
# Success (last 7d)
qry_credential_success_7d = '''
search SessionResult where success and time_index > (currentTime() - 7*24*3600*10000000)
show (credential or slave) as 'SessionResult.credential_or_slave',
     (credential or slave) as 'uuid',
     session_type as 'SessionResult.session_type',
     outpost as 'SessionResult.outpost'
processwith countUnique(1,0)
'''
cols = ["SessionResult.credential_or_slave","uuid","SessionResult.session_type","SessionResult.outpost","Count"]
suxCreds7_prod = get_results(twprod, qry_credential_failure,cols)
print(twprod['target'])
display(suxCreds7_prod.head(5))
suxCreds7_dev = get_results(twdev, qry_credential_failure,cols)
print(twdev['target'])
display(suxCreds7_dev.head(5))

In [ ]:
# DeviceInfo success (last 7d)
qry_deviceinfo_success_7d = '''
search DeviceInfo where method_success
  and nodecount(traverse DiscoveryResult:DiscoveryAccessResult:DiscoveryAccess:DiscoveryAccess
                traverse DiscoveryAccess:Metadata:Detail:SessionResult) = 0
  and time_index > (currentTime() - 7*24*3600*10000000)
show (last_credential or last_slave) as 'DeviceInfo.last_credential',
     (last_credential or last_slave) as 'uuid',
     access_method as 'DeviceInfo.access_method'
process with countUnique(1,0)
'''
cols = ["DeviceInfo.last_credential","uuid","DeviceInfo.access_method","Count"]
suxDev7_prod = get_results(twprod, qry_deviceinfo_success_7d,cols)
print(twprod['target'])
display(suxDev7_prod.head(5))
suxDev7_dev = get_results(twdev, qry_deviceinfo_success_7d,cols)
print(twdev['target'])
display(suxDev7_dev.head(5))

In [ ]:
# Credential failure (last 7d)
qry_credential_failure_7d = '''
search SessionResult where not success and time_index > (currentTime() - 7*24*3600*10000000)
show (credential or slave) as 'SessionResult.credential_or_slave',
     (credential or slave) as 'uuid',
     session_type as 'SessionResult.session_type',
     outpost as 'SessionResult.outpost'
processwith countUnique(1,0)
'''
cols = ["SessionResult.credential_or_slave","uuid","SessionResult.session_type","SessionResult.outpost","Count"]
failCreds7_prod = get_results(twprod, qry_credential_failure_7d, cols)
print(twprod['target'])
display(failCreds7_prod.head(5))
failCreds7_dev = get_results(twdev, qry_credential_failure_7d, cols)
print(twdev['target'])
display(failCreds7_dev.head(5))

## Build the report rows

Loop through vault credentials, compute success/failure counts and percentages,
attach scheduling/exclusion coverage, and outpost info.

In [ ]:
def parse_ranges(ranges):
    """
    Accepts a string like '10.0.0.0/8,192.168.0.0/16,::/0' or a list of strings.
    Returns a list of ip_network objects, skipping invalids.
    """
    if ranges is None or (isinstance(ranges, float) and pd.isna(ranges)):
        return []

    if isinstance(ranges, str):
        parts = [p.strip() for p in ranges.replace(';', ',').split(',') if p.strip()]
    elif isinstance(ranges, list):
        parts = []
        for r in ranges:
            if r is None or (isinstance(r, float) and pd.isna(r)):
                continue
            parts.extend([p.strip() for p in str(r).replace(';', ',').split(',') if p.strip()])
    else:
        parts = [str(ranges).strip()]

    # normalise common typos
    norm = []
    for p in parts:
        if p == '::0':
            p = '::/0'
        norm.append(p)

    nets = []
    for p in norm:
        try:
            nets.append(ip_network(p, strict=False))
        except Exception:
            # skip malformed/non-CIDR entries
            continue
    return nets

def to_rows(entries):
    """
    Converts post_search(...) output to a list of row dicts using the first block's headings/results.
    Accepts a DataFrame, list[dict], or raw dict.
    """
    if isinstance(entries, pd.DataFrame):
        # if they gave us a df of the 'first page', just return its records
        return entries.replace({pd.NA: None}).where(pd.notna(entries), None).to_dict(orient='records')

    if isinstance(entries, list) and entries and isinstance(entries[0], dict):
        block = entries[0]
        if 'headings' in block and 'results' in block:
            df = pd.DataFrame(block['results'], columns=block['headings'])
            return df.replace({pd.NA: None}).where(pd.notna(df), None).to_dict(orient='records')

    if isinstance(entries, dict) and 'headings' in entries and 'results' in entries:
        df = pd.DataFrame(entries['results'], columns=entries['headings'])
        return df.replace({pd.NA: None}).where(pd.notna(df), None).to_dict(orient='records')

    # fallback: assume it's already list[dict]
    return entries if isinstance(entries, list) else []

def labels_covering_ranges(entries, cred_ranges):
    """
    Returns sorted unique labels whose Scan_Range overlaps any of the cred_ranges networks.
    entries: post_search(...) result or a DataFrame/list[dict] with columns: Label, Scan_Range
    cred_ranges: string '10.0.0.0/8,::/0' or list of CIDR strings
    """
    labels = []
    cred_nets = parse_ranges(cred_ranges)
    if not cred_nets:
        return labels

    rows = to_rows(entries)
    for row in rows:
        label = row['Label']
        scan_rs = row['Scan_Range']  # may be str or list
        scan_nets = parse_ranges(scan_rs)
        if not scan_nets or not label:
            continue

        # overlap check (IPv4 vs IPv6 mismatches are naturally non-overlapping here)
        found = False
        for cn in cred_nets:
            for sn in scan_nets:
                # only compare same address family
                if cn.version != sn.version:
                    continue
                if cn.overlaps(sn):
                    found = True
                    break
            if found:
                break

        if found:
            labels.append(label)

    return sorted(set(labels))

def build_cred_report(
    vault_creds,
    suxCreds, suxDev, failCreds,
    suxCreds7, suxDev7, failCreds7,
    scan_ranges, exclude_ranges,
    cred_outpost_map
):
    rows = []

    for cred in vault_creds or []:
        print(cred)
        if not isinstance(cred, dict):
            print("Not instance, skipping")
            continue

        idx = cred["index"]
        uuid = (cred["uuid"] or "").strip()
        print(f"checking uuid: {uuid}")
        if not uuid:
            print("No UUID, skipping")
            continue

        uuid_key = uuid.split("/")[-1].lower()
        label = cred["label"]
        enabled = bool(cred["enabled"])
        types = cred["types"]
        usage = cred["usage"]

        # Best-effort username resolution
        username = (
            cred.get("username")
            or cred.get("snmp.v3.securityname")
            or cred.get("aws.access_key_id")
            or cred.get("azure.application_id")
        )

        ip_range = cred["ip_range"]
        ip_exclusion = cred["ip_exclusion"]
        status = "Enabled" if enabled else "Disabled"

        # Lookup from maps
        sessions  = suxCreds.get(uuid_key,  [None, 0])
        devinfos  = suxDev.get(uuid_key,    [None, 0])
        failure   = failCreds.get(uuid_key, [None, 0])
        sessions7 = suxCreds7.get(uuid_key, [None, 0])
        devinfos7 = suxDev7.get(uuid_key,   [None, 0])
        failure7  = failCreds7.get(uuid_key,[None, 0])

        # Active if present in any mapping or any count present
        active = (
            uuid_key in suxCreds or uuid_key in suxDev or uuid_key in failCreds or
            uuid_key in suxCreds7 or uuid_key in suxDev7 or uuid_key in failCreds7 or
            any(x[1] for x in [sessions, devinfos, failure, sessions7, devinfos7, failure7])
        )

        # Success/failure counts
        success_all = int(sessions[1] or 0) + int(devinfos[1] or 0)
        fails_all   = int(failure[1] or 0)
        total       = success_all + fails_all
        percent_all = (success_all / total) if total else 0.0

        success7 = int(sessions7[1] or 0) + int(devinfos7[1] or 0)
        fails7   = int(failure7[1] or 0)
        total7   = success7 + fails7
        percent7 = (success7 / total7) if total7 else 0.0

        # Scan coverage
        scheduled_scans = labels_covering_ranges(scan_ranges, ip_range)
        excluded_scans  = labels_covering_ranges(exclude_ranges, ip_range)

        # Outpost info
        op_info    = cred_outpost_map.get(uuid_key, {})
        outpost_id = op_info.get("id")
        outpost_url= op_info.get("url")

        proto = sessions[0] or failure[0] or types

        if active:
            rows.append([
                label, idx, uuid, username, proto,
                success_all, fails_all, percent_all, percent7,
                status, usage, ip_range, ip_exclusion,
                scheduled_scans or None, excluded_scans or None,
                outpost_id, outpost_url
            ])
        else:
            rows.append([
                label, idx, uuid, username, types,
                0, 0, 0.0, 0.0,
                f"Credential appears to not be in use ({status})", usage,
                ip_range, ip_exclusion,
                scheduled_scans or None, excluded_scans or None,
                outpost_id, outpost_url
            ])

    # Build DataFrame
    return pd.DataFrame(rows, columns=[
        "label", "index", "uuid", "username", "protocol",
        "success_all", "fails_all", "percent_all", "percent7",
        "status", "usage", "ip_range", "ip_exclusion",
        "scheduled_scans", "excluded_scans",
        "outpost_id", "outpost_url"
    ])

headers = [
    'Discovery Instance', 'Credential', 'Index', 'UUID', 'Login ID', 'Protocol',
    'Successes', 'Failures', 'Success % All Time', 'Success % 7 Days', 'State',
    'Usage', 'Ranges', 'Excludes', 'Scheduled Scans', 'Exclusion Lists',
    'Outpost', 'Outpost URL'
]

df_out_prod = build_cred_report(
    vault_prod,
    suxCreds_prod, suxDev_prod, failCreds_prod,
    suxCreds7_prod, suxDev7_prod, failCreds7_prod,
    scan_prod, ex_prod,
    prod_map
)

# Build the dev report as well (you never created this)
df_out_dev = build_cred_report(
    vault_dev,
    suxCreds_dev, suxDev_dev, failCreds_dev,
    suxCreds7_dev, suxDev7_dev, failCreds7_dev,
    scan_dev, ex_dev,
    dev_map
)

print("\nProd preview:")
display(df_out_prod.head(5))
print("\nDev preview:")
display(df_out_dev.head(5))

## Save to CSV

Writes the report to the standard output folder as used by the CLI.

In [ ]:
OUTPUT_CSV = str(twprod['output_dir'] / 'credential_success.csv')
df_out_prod.to_csv(OUTPUT_CSV, index=False)
print(f'Saved to {OUTPUT_CSV}')

OUTPUT_CSV = str(twdev['output_dir'] / 'credential_success.csv')
df_out_dev.to_csv(OUTPUT_CSV, index=False)
print(f'Saved to {OUTPUT_CSV}')

End